# Load profiles and save it as a parquet file

## Import libraries

In [7]:
import io
import pandas as pd
import plotly.express as px
import plotly.io as pio

## Helper functions

In [8]:
profile_formatter = (
    "s3://cellpainting-gallery/cpg0016-jump/"
    "{Metadata_Source}/workspace/profiles/"
    "{Metadata_Batch}/{Metadata_Plate}/{Metadata_Plate}.parquet"
)

## Load metadata

In [9]:
plates = pd.read_csv("../../datasets/metadata/plate.csv.gz")
wells = pd.read_csv("../../datasets/metadata/well.csv.gz")
compound = pd.read_csv("../../datasets/metadata/compound.csv.gz")
orf = pd.read_csv("../../datasets/metadata/orf.csv.gz")


## Load plates

In [10]:
sample = (
    plates.query('Metadata_Source=="source_4"')
    .query('Metadata_PlateType=="ORF"')
    .query('Metadata_Batch=="2021_07_12_Batch8"') # drop this later
)

# count the number of plates per batch
sample.groupby("Metadata_Batch")["Metadata_Plate"].count()


Metadata_Batch
2021_07_12_Batch8    20
Name: Metadata_Plate, dtype: int64

## Loading profiles

Now let's load the profiles from these plates.

Setting `columns = None` below will load all of the features.

<div class="alert alert-warning">
WARNING: Files are located in S3. This loop loads only two features per each sampled plate; loading many feature and/or many plates can take several minutes.
</div>

In [11]:
dframes = []
columns = [
    "Metadata_Source",
    "Metadata_Plate",
    "Metadata_Well",
    "Cells_AreaShape_Eccentricity",
    "Nuclei_AreaShape_Area",
]
for _, row in sample.iterrows():
    s3_path = profile_formatter.format(**row.to_dict())
    dframes.append(
        pd.read_parquet(s3_path, storage_options={"anon": True}, columns=columns)
    )
dframes = pd.concat(dframes)

dframes.groupby("Metadata_Plate")["Metadata_Well"].count()


Metadata_Plate
BR00124787    384
BR00124788    384
BR00125619    383
BR00125620    384
BR00125621    384
BR00125622    384
BR00125623    384
BR00125624    384
BR00125625    384
BR00125626    384
BR00125627    384
BR00125628    384
BR00125629    384
BR00125630    384
BR00125631    384
BR00125633    384
BR00125634    384
BR00125635    384
BR00125636    384
BR00125637    384
Name: Metadata_Well, dtype: int64

Join features with metadata


In [12]:
metadata = orf.merge(wells, on="Metadata_JCP2022")
ann_dframe = metadata.merge(
    dframes, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"]
)

ann_dframe.groupby("Metadata_Plate")["Metadata_Well"].count()


Metadata_Plate
BR00124787    368
BR00124788    368
BR00125619    367
BR00125620    368
BR00125621    368
BR00125622    368
BR00125623    368
BR00125624    368
BR00125625    368
BR00125626    368
BR00125627    368
BR00125628    368
BR00125629    368
BR00125630    368
BR00125631    368
BR00125633    368
BR00125634    368
BR00125635    368
BR00125636    368
Name: Metadata_Well, dtype: int64

Print sample of rows from `ann_dframe` (only `Metadata_` columns)

In [13]:
ann_dframe.filter(regex="^Metadata_").sample(5)

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,Metadata_Insert_Length,Metadata_pert_type,Metadata_Source,Metadata_Plate,Metadata_Well
2580,JCP2022_905168,ccsbBroad304_05525,ORF012541.1_TRC304.1,pLX_304,NM_181727.2,SPATA12,353324,9606,spermatogenesis associated 12,100.0,570.0,trt,source_4,BR00125624,O06
4058,JCP2022_908778,ccsbBroad304_09405,ORF008924.1_TRC304.1,pLX_304,NM_031910.4,C1QTNF6,114904,9606,C1q and TNF related 6,99.2,834.0,trt,source_4,BR00125630,B20
5523,JCP2022_912399,ccsbBroad304_13306,ORF011423.1_TRC304.1,pLX_304,NM_139246.5,TSTD2,158427,9606,thiosulfate sulfurtransferase like domain cont...,94.5,1467.0,trt,source_4,BR00125621,A12
4291,JCP2022_909451,ccsbBroad304_10152,ORF013171.1_TRC304.1,pLX_304,NR_149714.1,C9orf106,414318,9606,chromosome 9 open reading frame 106 (putative),NaN,696.0,trt,source_4,BR00125626,A20
3604,JCP2022_907592,ccsbBroad304_08117,ORF009020.1_TRC304.1,pLX_304,NM_013314.3,BLNK,29760,9606,B cell linker,1.0,1322.0,trt,source_4,BR00125626,P05


## Save profiles

In [14]:
ann_dframe.to_parquet("output/profiles.parquet", index=False)

In [17]:
# anti join `Metadata_JCP2022` to get the rows in `wells` that are not in `orf`
wells[~wells.Metadata_JCP2022.isin(orf.Metadata_JCP2022)]



,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_JCP2022
0,source_1,UL000081,A02,JCP2022_033924
1,source_1,UL000081,A03,JCP2022_085227
2,source_1,UL000081,A04,JCP2022_033924
3,source_1,UL000081,A05,JCP2022_047857
4,source_1,UL000081,A06,JCP2022_072229
...,...,...,...,...
1096069,source_9,GR00004421,Z44,JCP2022_999999
1096070,source_9,GR00004421,Z45,JCP2022_999999
1096071,source_9,GR00004421,Z46,JCP2022_999999
1096072,source_9,GR00004421,Z47,JCP2022_033924
